In [ ]:
TRAIN_DATA = "DATA/train.csv"
TEST_DATA = "DATA/test.csv"
SUBMISSON_DATA = "DATA/submission.csv"

In [ ]:
import pandas as pd
import statsmodels.api as sm

In [ ]:
df = pd.read_csv(TEST_DATA)
df

In [ ]:
df.rename({df.columns[9]: "Решение"}, inplace=True)
df.columns

In [ ]:
tdf = df[df['Статус'] == 'Отменен']
tdf

In [ ]:
priority = {'3-Низкий': 0, '2-Средний': 1, '1-Высокий': 2, '0-Критический': 3}
status = {'Отменен': 0, 'Закрыт': 1}
obr = {'Запрос': 0, 'Инцидент': 1}
critical = {'4-Нет': 0, "3-Базовая": 2, "2-Повышенная": 3, "1-Особая": 1}
influence = {"4-Нет влияния": 0, "3-Малое": 1, "2-Значительное": 2, "1-Всеохватывающее": 3}

In [ ]:
df['Приоритет'] = df['Приоритет'].map(priority)
df

In [ ]:
df['Статус'] = df['Статус'].map(status)
df

In [ ]:
df['Тип обращения на момент подачи'] = df['Тип обращения на момент подачи'].map(obr)
df['Тип обращения итоговый'] = df['Тип обращения итоговый'].map(obr)
df

In [ ]:
df['Критичность'] = df['Критичность'].map(critical)
df['Влияние'] = df['Влияние'].map(influence)
df

In [ ]:
len(df)

In [ ]:
df['Дата обращения'] = pd.to_datetime(df['Дата обращения'])
df['Дата закрытия обращения'] = pd.to_datetime(df['Дата закрытия обращения'])
df['range_time'] = df['Дата закрытия обращения'] - df['Дата обращения']
df['Крайний срок'] = pd.to_datetime(df['Крайний срок'])
df['time_to_solve'] = (df['Крайний срок'] - df['Дата обращения'])
df

In [ ]:
df['range_time'] = df['range_time'].dt.total_seconds() / 3600
df['time_to_solve'] = df['time_to_solve'].dt.total_seconds() / 3600
df

In [ ]:
df['range_time'] = df['range_time'].fillna(-1)

In [ ]:
system_l = df['Система'].unique()
system_d = {}
for i in range(len(system_l)):
    system_d[system_l[i]] = i
system_d

In [ ]:
df['Система'] = df['Система'].map(system_d)
df

In [ ]:
df['koef'] = df['Критичность'] * df['Влияние']
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Содержание'])
df['numeric_sod'] = labels
df

In [ ]:
place_l = df['Место'].unique()
place_d = {}
for i in range(len(place_l)):
    place_d[place_l[i]] = i
df['Место'] = df['Место'].map(place_d)
df

In [ ]:
func_l = df['Функциональная группа'].unique()
func_d = {}
for i in range(len(func_l)):
    func_d[func_l[i]] = i
df['Функциональная группа'] = df['Функциональная группа'].map(func_d)
df

In [ ]:
label_encoder2 = LabelEncoder()
labels2 = label_encoder2.fit_transform(df[df.columns[9]])
df['numeric_ans'] = labels2
df

In [ ]:
df.to_csv("DATA/new_test.csv", index=False)

In [ ]:
target = df['Тип обращения итоговый']
features = df[["numeric_sod", "numeric_ans",'Приоритет', 'Тип обращения на момент подачи',
               'Критичность', 'Влияние', 'range_time', 'Система', 'Место',
               'Функциональная группа', 'koef', 'time_to_solve']]
features = sm.add_constant(features)
model = sm.OLS(target, features)
result = model.fit()
model2 = sm.Logit(target, features)
result2 = model2.fit()
print(result.summary())

In [ ]:
print(result2.summary())

In [ ]:
# Визуализация данных, поиск зависимостей
import seaborn as sns

In [ ]:
sns.displot(df, x="Влияние",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
sns.displot(df[df['Влияние']==2 ], x="Приоритет",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
sns.displot(df[df['Влияние']==2 ], x="Критичность",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
df['koef'] = (df['Приоритет'] + df['Критичность'])/df['Влияние']
df

In [ ]:
df[df['Критичность'] == 3]

In [ ]:
sns.displot(df, x="range_time",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
import json

with open("DATA/place_test.json", 'w', encoding='utf-8') as f:
    f.write(json.dumps(place_d, ensure_ascii=False))

with open("DATA/system_test.json", 'w', encoding='utf-8') as f:
    f.write(json.dumps(system_d, ensure_ascii=False))

with open("DATA/func_test.json", 'w', encoding='utf-8') as f:
    f.write(json.dumps(func_d, ensure_ascii=False))